# BERT

## Requirements

In [375]:
#Imports
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

import _helpers as hp

## Load Data

In [376]:
ASR_data = pd.read_csv("snips/new_ASR_with_labels.csv") # ASR data with improved speech recognition with 15555 framerate and with autocorrection applied

GT_data = pd.read_csv("snips/merged_GT_data.csv") # Groundtruth data

## Import pre-trained DistilBERT model and tokenizer

DistilBERT is a small, fast, cheap and light Transformer model. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT's performances

In [377]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Import pre-trained BERT model and tokenizer

If we want to use BERT instead of DistilBERT we comment the previous model importation and use this one instead:

In [378]:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-small-uncased')

In [379]:
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/home/goncalo/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Tokenization

In [380]:
tokenized_GT_data = GT_data["transcript"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

tokenized_ASR_data = ASR_data["transcript"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

The problem now is that the tokenized vectors ar not with the name size

In [381]:
if  max(tokenized_GT_data.apply(len)) >  max(tokenized_ASR_data.apply(len)):
    _max = max(tokenized_GT_data.apply(len))
    _min = min(tokenized_GT_data.apply(len))
else:
    _max = max(tokenized_ASR_data.apply(len))
    _min = min(tokenized_ASR_data.apply(len))

print("MX:",_max)
print("MIN:",_min)

MX: 22
MIN: 3


lets fix that with a simple padding function

In [382]:
padded_GT_data = np.array(list(tokenized_GT_data.apply(lambda x: hp.padding_func(x,_max))))
padded_ASR_data = np.array(list(tokenized_ASR_data.apply(lambda x: hp.padding_func(x,_max))))
print(padded_GT_data.shape)
print(padded_ASR_data.shape)

(2224, 22)
(1660, 22)


# Masking

If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [383]:
attention_mask_GT = np.where(padded_GT_data != 0, 1, 0)

attention_mask_ASR = np.where(padded_ASR_data != 0, 1, 0)

# Processing with DistrilBert

In [384]:
input_ids_GT = torch.tensor(padded_GT_data)  
attention_mask_GT = torch.tensor(attention_mask_GT)

with torch.no_grad():
    last_hidden_states_GT = model(input_ids_GT, attention_mask=attention_mask_GT)

In [385]:
input_ids_ASR = torch.tensor(padded_ASR_data)  
attention_mask_ASR = torch.tensor(attention_mask_ASR)

with torch.no_grad():
    last_hidden_states_ASR = model(input_ids_ASR, attention_mask=attention_mask_ASR)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence

basically [CLS] contains all information of the sentence and representing the sentence-level classification.

In [404]:
#Taking the CLS features of each sentence
features_GT = last_hidden_states_GT[0][:,0,:].numpy() 
features_GT.shape

(2224, 768)

In [387]:
#Taking the CLS features of each sentence
features_ASR = last_hidden_states_ASR[0][:,0,:].numpy() 
features_ASR.shape

(1660, 768)

In [388]:
labels_GT = GT_data["user_action_num"]
labels_ASR = ASR_data["user_action_num"]

# Classifiers

## Train/Test Split

In [389]:
# Split into training and test data

train_features, test_features, train_labels, test_labels = train_test_split(features_ASR, labels_ASR, train_size= 0.9)

For train we will use both the training set of ASR data plus all the groudtruth data in order to have more data and build more robust models

In [390]:
train_features = np.concatenate((train_features,features_GT), axis=0)
train_features.shape

(3718, 768)

In [391]:
train_labels = np.concatenate((train_labels,labels_GT), axis=0)
train_labels.shape

(3718,)

## Logistic Regression

In [392]:
lgr = LogisticRegression(max_iter= 1000) #Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr) #Scale feature space
lgr_pipe.fit(train_features, train_labels)


lgr_pred_labels = lgr_pipe.predict(test_features) #predictions

lgr_score = lgr_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [393]:
print(classification_report(test_labels, lgr_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.71      0.79      0.75        19
     SwitchLightOn       1.00      0.90      0.95        31
IncreaseBrightness       0.88      0.86      0.87        35
DecreaseBrightness       0.73      0.83      0.78        23
SetLightBrightness       0.75      0.89      0.81        27
     SetLightColor       0.96      0.77      0.86        31

          accuracy                           0.84       166
         macro avg       0.84      0.84      0.84       166
      weighted avg       0.86      0.84      0.85       166

[[15  0  1  1  1  1]
 [ 1 28  0  1  1  0]
 [ 1  0 30  2  2  0]
 [ 1  0  1 19  2  0]
 [ 1  0  0  2 24  0]
 [ 2  0  2  1  2 24]]

ACCURACY: 0.8433734939759037


# Naive Bayes

### Gaussian Naive Bayes

In [394]:
gnb = GaussianNB() #Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb) #Scale feature space
gnb_pipe.fit(train_features, train_labels)


gnb_pred_labels = gnb_pipe.predict(test_features) #predictions

gnb_score = gnb_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [395]:
print(classification_report(test_labels, gnb_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.39      0.37      0.38        19
     SwitchLightOn       0.33      0.58      0.42        31
IncreaseBrightness       0.68      0.37      0.48        35
DecreaseBrightness       0.40      0.35      0.37        23
SetLightBrightness       0.66      0.78      0.71        27
     SetLightColor       0.87      0.65      0.74        31

          accuracy                           0.52       166
         macro avg       0.56      0.52      0.52       166
      weighted avg       0.58      0.52      0.53       166

[[ 7 10  1  1  0  0]
 [ 6 18  1  1  3  2]
 [ 3  9 13  5  5  0]
 [ 2  7  3  8  2  1]
 [ 0  1  1  4 21  0]
 [ 0  9  0  1  1 20]]

ACCURACY: 0.5240963855421686


## SVM

In [396]:
svm = SVC() #Create the classification model

svm_pipe = make_pipeline(preprocessing.StandardScaler(), svm) #Scale feature space
svm_pipe.fit(train_features, train_labels)


svm_pred_labels = svm_pipe.predict(test_features) #predictions

svm_score = svm_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [397]:
print(classification_report(test_labels, svm_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)


                    precision    recall  f1-score   support

    SwitchLightOff       0.75      0.79      0.77        19
     SwitchLightOn       0.78      0.81      0.79        31
IncreaseBrightness       0.88      0.83      0.85        35
DecreaseBrightness       0.72      0.78      0.75        23
SetLightBrightness       0.75      0.89      0.81        27
     SetLightColor       0.96      0.74      0.84        31

          accuracy                           0.81       166
         macro avg       0.81      0.81      0.80       166
      weighted avg       0.82      0.81      0.81       166

[[15  2  0  2  0  0]
 [ 2 25  1  1  2  0]
 [ 1  2 29  1  2  0]
 [ 0  2  1 18  2  0]
 [ 0  0  0  2 24  1]
 [ 2  1  2  1  2 23]]

ACCURACY: 0.8072289156626506


## NEURAL NETWORK

In [398]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100), activation='relu', solver='adam', max_iter=5000) #Create the classification model

mlp_pipe = make_pipeline(preprocessing.StandardScaler(), mlp) #Scale feature space
mlp_pipe.fit(train_features, train_labels)


mlp_pred_labels = mlp_pipe.predict(test_features) #predictions

mlp_score = mlp_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [399]:
print(classification_report(test_labels, mlp_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)


                    precision    recall  f1-score   support

    SwitchLightOff       0.71      0.89      0.79        19
     SwitchLightOn       0.93      0.90      0.92        31
IncreaseBrightness       0.84      0.77      0.81        35
DecreaseBrightness       0.83      0.83      0.83        23
SetLightBrightness       0.81      0.93      0.86        27
     SetLightColor       0.92      0.77      0.84        31

          accuracy                           0.84       166
         macro avg       0.84      0.85      0.84       166
      weighted avg       0.85      0.84      0.84       166

[[17  1  1  0  0  0]
 [ 1 28  0  0  1  1]
 [ 3  0 27  2  2  1]
 [ 2  0  0 19  2  0]
 [ 0  0  1  1 25  0]
 [ 1  1  3  1  1 24]]

ACCURACY: 0.8433734939759037


## Try your self

In [400]:
def user_friendly(sentence, mdl):
    """return action from sentence"""
    sent_token = tokenizer.encode(sentence, add_special_tokens=True)
    sent_pad = np.array([sent_token + [0]*(_max-len(sent_token))])
    sent_att_mask = np.where(sent_pad != 0, 1, 0)
    sent_input_ids = torch.tensor(sent_pad)  
    sent_attention_mask = torch.tensor(sent_att_mask)

    with torch.no_grad():
        sent_last_hidden_states = model(sent_input_ids, attention_mask=sent_attention_mask)
    
    feature = sent_last_hidden_states[0][:,0,:].numpy() 
    
    prediction = mdl.predict(feature)
    user_action = hp.indx2action(prediction)
    return user_action

In [401]:
user_friendly("red light to the room", lgr_pipe)

['SwitchLightOff']